# Implementación del método de la máxima verosimilitud para la regresión logística.

## Definir la función de entorno L(b)

$ L(\beta) = \sum{i=1}^n P_i^{y_i}(1-Pi)^{1-y_i} $

In [1]:
def likelihood (y, pi):
    import numpy as np
    total_sum = 1
    sum_in = list(range(1, len(y)+1))
    for i in range(len(y)):
        sum_in[i] = np.where(y[i]==1, pi[i], 1-pi[i])
        total_sum = total_sum * sum_in[i]
    return total_sum

### Calcular las probabilidades para cada observación

$ L(\beta) = P_i = P(x_i) = \frac{1}{1 + e^ {-\sum_{j=0}^k(\beta_j \cdot X_{ij})}} $

In [2]:
def logitprobs(X, beta):
    import numpy as np
    n_rows = np.shape(X)[0]
    n_cols = np.shape(X)[1]
    pi = list(range (1, n_rows+1))
    expon = list(range(1, n_rows+1))
    for i in range (n_rows): # Este for es para las i, o sea para las probabilidades
        expon[i] = 0
        for j in range(n_cols):  # Calculo el sumatorio de las k columnas, y saco el exponente
            ex = X[i][j] * beta[j] #Producto de los exponentes Beta_j y X_ij
            expon[i] = ex + expon[i] #Suma total de los exponentes
        with np.errstate(divide="ignore", invalid="ignore"):
            pi[i]=1/(1+np.exp(-expon[i]))
    return pi
            

### Calcular la matriz diagonal W

$ \omega_{(\beta)} = diag(P_i \cdot (1-P_i))_{i=1}^{n} $

In [3]:
def findW(pi):
    import numpy as np
    n = len(pi)
    W = np.zeros(n*n).reshape(n,n) #Creo un vector de 0s  y luego una matriz de n x n de ceros con reshape
    for i in range (n):
        print(i)
        W[i, i] = pi[i]*(1-pi[i])
        W[i, i].astype(float)
    return W

### Definir la función logística

$ \Delta\beta = \frac{f(\beta)}{f'(\beta)} $

$ f(\beta) = \frac{\delta l}{\delta \beta} = X \cdot (y-P(\beta)) $

$ f'(\beta) = \frac{\delta^2 l}{\delta \beta^2} = X \cdot \omega_(\beta) \cdot X^t  $

In [34]:
def logistics (X, Y, limit): #El límite del cambio de un paso al siguiente hasta que converja
    from numpy import linalg
    nrow = np.shape(X)[0]
    bias = np.ones(nrow).reshape(nrow, 1)
    X_new = np.append (X, bias, axis = 1) #Agrego una columna de unos a la derecha del los datos, aux.
    ncol = np.shape(X_new)[1] #Número de columnas en la posición 1 porque tendrá una más que antes.
    beta = np.zeros(ncol).reshape(ncol, 1) #Toda una col de 0 de la misma lon que e ncol, apiladas
    root_dif = np.array(range(1,ncol+1)).reshape(ncol,1) #Diferencias entre las raíces / los betas.
    iter_i = 10000
    counter = 0
    while (iter_i>limit):
        print("El producto de iteraciones es iter_i: " + str(iter_i) + ", El límite es: " + str(limit))
        pi = logitprobs(X_new, beta)
        print ("El valor de las probabilidades parciales Pi es: " + str(pi))
        W = findW(pi)
        print("La matriz es W: " + str(W))
        #num y den, que son numerador y denominador se transponen para poder ser multiplicadas.
        num = (np.transpose(np.matrix(X_new))*np.matrix(Y-np.transpose(pi)).transpose())
        den = (np.matrix(np.transpose(X_new))*np.matrix(W)*np.matrix(X_new))
        root_dif = np.array(linalg.inv(den)*num) #Acá está el delta beta calculado
        beta = beta + root_dif
        print("El valor de beta es: " + str(beta))
        iter_i = np.sum(root_dif*root_dif)
        ll = likelihood(Y, pi) #Factor de verosimilitud
        counter += 1
        print("El n° iteraciones a la que converge es :" + str(counter))
    return beta

### Comprobación experimental

In [35]:
import numpy as np

In [36]:
X = np.array(range(10)).reshape(10,1)
X

array([[0],
       [1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8],
       [9]])

In [37]:
Y = [0,0,0,0,1,0,1,0,1,1]

In [38]:
bias = np.ones(10).reshape(10,1)
X_new = np.append(X, bias, axis = 1)

In [39]:
X_new

array([[0., 1.],
       [1., 1.],
       [2., 1.],
       [3., 1.],
       [4., 1.],
       [5., 1.],
       [6., 1.],
       [7., 1.],
       [8., 1.],
       [9., 1.]])

In [41]:
a = logistics(X, Y, 0.00001)

El producto de iteraciones es iter_i: 10000, El límite es: 1e-05
El valor de las probabilidades parciales Pi es: [array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5]), array([0.5])]
0
1
2
3
4
5
6
7
8
9
La matriz es W: [[0.25 0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.25 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.25 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.25 0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.25 0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.25 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.25 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.25]]
El valor de beta es: [[ 0.43636364]
 [-2.36363636]]
El n° iteraciones a la que converge es :1
El producto de iteraciones es iter_i: 5.777190082644626, El límite es: 1e-05
El valor

La ecuación conchasumadre es: Y =  0.66220827 -3.69557172X esto es lo que vendría siendo el 
exponente de la función logística:

$ P = \frac{1}{1+e^{-(0.66220827-3.69557172\cdot X}} $

In [11]:
ll = likelihood (Y, logitprobs(X,a))

In [12]:
ll #La suma total

array([1.32622426e-06])

## Con el paquete statsmodel de python

In [13]:
import statsmodels.api as sm

In [14]:
logit_model = sm.Logit(Y, X_new)

In [15]:
result = logit_model.fit()

Optimization terminated successfully.
         Current function value: 0.431012
         Iterations 6


In [43]:
print(result.summary2())

                        Results: Logit
Model:              Logit            Pseudo R-squared: 0.360   
Dependent Variable: y                AIC:              12.6202 
Date:               2022-03-24 15:30 BIC:              13.2254 
No. Observations:   10               Log-Likelihood:   -4.3101 
Df Model:           1                LL-Null:          -6.7301 
Df Residuals:       8                LLR p-value:      0.027807
Converged:          1.0000           Scale:            1.0000  
No. Iterations:     6.0000                                     
-----------------------------------------------------------------
          Coef.    Std.Err.      z      P>|z|     [0.025   0.975]
-----------------------------------------------------------------
x1        0.6622     0.4001    1.6551   0.0979   -0.1220   1.4464
const    -3.6956     2.2889   -1.6145   0.1064   -8.1818   0.7906



Como podemos ver en el summary x1 es el eje de coordenadas y const el factor de multiplicación y

estos coinciden exactamente con nuestro modelo, por lo que podemos decir es que nuestro porgrama

está bien hecho y la aproximación de Newton - Rapson están bien calculados.